In [1]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from dateutil import parser
from dateutil.relativedelta import relativedelta
from collections import Counter
import math

# Open and Pre-process

In [2]:
data_location = 'data'
outlooks, pph, reports = read_datasets(data_location)

reading outlooks 1
reading outlooks 2


c:\Users\miles\OneDrive\Documents\UW\Research\utils_filter.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/' + mod_string + '_outlooks_2.shp'))


reading pph
reading storm reports


In [ ]:
tz_conversions = {'PST': timedelta(hours=8),
                  'MST': timedelta(hours=7),
                  'CST': timedelta(hours=6),
                  'CSt': timedelta(hours=6),
                  'CSC': timedelta(hours=6),
                  'SCT': timedelta(hours=6),
                  'EST': timedelta(hours=5),
                  'ESt': timedelta(hours=5),
                  'PDT': timedelta(hours=7),
                  'MDT': timedelta(hours=6),
                  'CDT': timedelta(hours=5),
                  'EDT': timedelta(hours=4),
                  'HST': timedelta(hours=10),
                  'SST': timedelta(hours=11),
                  'GST': timedelta(hours=10),
                  'AKS': timedelta(hours=9),
                  'AST': timedelta(hours=4),
                  'UNK': timedelta(hours=5),
                  'GMT': timedelta(0)}

def get_reports_date_strings(date_times, timezones):
    # returns list of strings of date of given datetime and timezone (where day cutoffs are 12-12 UTC) formatted as 'YYYYMMDD0000'
    for datetime, timezone, i in zip(date_times, timezones, range(len(timezones))):
        #print(datetime + ' ' + timezone[:3])
        datetime = parser.parse(datetime)
        datetime = datetime + tz_conversions[timezone[:3]]
        #print(datetime)
        if (datetime.hour < 12):
            datetime = datetime - timedelta(days = 1)
        if datetime.year > 2049:
            datetime = datetime - relativedelta(years = 100)
        datetime = datetime.strftime("%Y%m%d") + '0000'
        if i == 0:
            ret = [datetime]
        else:
            ret.append(datetime)
    return ret

def get_pph_date_strings(times):
    # returns a list of strings of given dates formatted as 'YYYYMMDD0000'
    for datetime, i in zip(times, range(len(times))):
        string = datetime.dt.strftime("%Y%m%d").values + '0000'
        if i == 0:
            ret = [string]
        else:
            ret.append(string)
    return ret

In [ ]:
reports['DATE'] = get_reports_date_strings(reports['BEGIN_DATE_TIME'], reports['CZ_TIMEZONE']) 
pph['time'] = get_pph_date_strings(pph.time) 
# subset outlooks into only one day 1, two day 2, and one day 3 categorical outlooks 
# day 3: cycle not -1. day 2: cycle not -1. Day 1: cycle 6. Category: categorical. 
#outlooks = outlooks[(((outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6)) | ((outlooks['DAY'] == 2) & (outlooks['CYCLE'] != -1)) | ((outlooks['DAY'] == 3) & (outlooks['CYCLE'] != -1)))
#        & (outlooks['CATEGORY'] == 'CATEGORICAL')]

# reset incicies
outlooks = outlooks.reset_index(drop=True)
reports = reports.drop(columns=['geometry'])

# Gridize outlooks

,ISSUE,EXPIRE,PRODISS,TYPE,DAY,THRESHOLD,CATEGORY,CYCLE,DATE,geometry
0,198701011200,198701021200,198701010635,C,1,TSTM,CATEGORICAL,-1,198701010000,"MULTIPOLYGON (((-80.69500 29.36500, -80.68200 ..."
1,198701011500,198701021200,198701011441,C,1,TSTM,CATEGORICAL,-1,198701010000,"POLYGON ((-70.15100 42.90900, -70.14600 42.899..."
2,198701011900,198701021200,198701011849,C,1,TSTM,CATEGORICAL,-1,198701010000,"POLYGON ((-73.73200 39.90700, -73.73200 39.892..."
3,198701021200,198701031200,198701020637,C,1,TSTM,CATEGORICAL,-1,198701020000,"POLYGON ((-124.43300 41.91800, -124.44100 41.9..."
5,198701021500,198701031200,198701021438,C,1,TSTM,CATEGORICAL,-1,198701020000,"MULTIPOLYGON (((-89.75800 28.96700, -89.75900 ..."
...,...,...,...,...,...,...,...,...,...,...
183177,202312310100,202312311200,202312310038,C,1,None,None,1,202312300000,None
183178,202312311200,202401011200,202312310520,C,1,None,None,6,202312310000,None
183181,202312311300,202401011200,202312311236,C,1,None,None,13,202312310000,None
183182,202312311630,202401011200,202312311558,C,1,None,None,16,202312310000,None


In [68]:
def get_outlooks_subset(outlooks, outlook_type):
    outlooks = outlooks[(outlooks['THRESHOLD'] != 'SIGN') & (outlooks['THRESHOLD'] != 'TSTM')]
    if outlook_type == 'Day 3':
        return outlooks[(outlooks['DAY'] == 3) & (outlooks['CYCLE'] != -1) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 2 7':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 2 17':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 1 Wind':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'WIND')]
    elif outlook_type == 'Day 1 Hail':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'HAIL')]
    elif outlook_type == 'Day 1 Tornado':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'TORNADO')]
    raise Exception("Invalid outlook_type given to get_outlooks_subset")

In [121]:
# as in mca: create_gridded_outlook_dataset. But will need to work overall, not just by hazard. day 3 is overall probability, days 1 and 2 by hazard
# day 3 use ANY SEVERE, day 2 use ANY SEVERE, day 1 construct from each hazard (assuming independence? or use highest prob, which basically assumes complete dependence? Is this what is done for day 2?)
# Things are entirely categorical before 2002, so even more reason to make that the cutoff.
save_location = 'data/outlooks/grid_outlooks.nc'

outlook_types = ['Day 3', 'Day 2 7', 'Day 2 17', 'Day 1 Wind', 'Day 1 Hail', 'Day 1 Tornado', 'Day 1']

outlook_dataset = xr.Dataset(
    data_vars=dict(
        lat=(['y', 'x'], pph['lat'].data),
        lon=(['y', 'x'], pph['lon'].data)
    ),
    coords=dict(
        time=(['time'], pph['time'].data),
        x=(['x'], pph['x'].data),
        y=(['y'], pph['y'].data),
        outlook=(['outlook'], outlook_types)
    ),
    attrs=dict(description="outlook as a percentage as a function of date, lat/lon, and which hazard type",
            grid = pph.grid),
)

outlook_dataset = outlook_dataset.assign(prob = (('time', 'y', 'x', 'outlook'), np.full((len(outlook_dataset['time']), len(outlook_dataset['y']), len(outlook_dataset['x']), len(outlook_types)), 0.0)))





Day 3
1980
1990
2000
2010
2020
Day 2 7
1980
1990
2000


KeyboardInterrupt: 

In [132]:

# for each outlook type
for outlook_type in outlook_types[1:-1]:
    print(outlook_type)
    outlooks_subset = get_outlooks_subset(outlooks, outlook_type)

    oldyear = 0
    array = np.zeros((len(outlook_dataset.time.values), len(outlook_dataset.y.values), len(outlook_dataset.x.values)))

    for date, i in zip(outlook_dataset.time.values, range(len(outlook_dataset.time.values))):
        lats = outlook_dataset.lat
        lons = outlook_dataset.lon
        datestring = str(date)
        datestring = datestring[0:4] + datestring[5:7] + datestring[8:10] + '0000'
        
        # printing to track progress
        year = datestring[0:4]
        if year != oldyear:
            print(year)
            oldyear = year

        outlooks_date = outlooks_subset[outlooks_subset['DATE'] == datestring]
        if (len(outlooks_date) > 0):
            for x, j in zip(outlook_dataset.x.values, range(len(outlook_dataset.x.values))):
                for y, k in zip(outlook_dataset.y.values, range(len(outlook_dataset.y.values))):
                    lat = lats.sel(x = x, y = y).values
                    lon = lons.sel(x = x, y = y).values
                    array[i, k, j] = find_threshold(outlooks_date, lat, lon)
    me = xr.DataArray(array, coords={'time': outlook_dataset.time.values, 'y': outlook_dataset.y.values, 'x': outlook_dataset.x.values},
                      dims=['time', 'y', 'x'])
    outlook_dataset['prob'].loc[dict(outlook = outlook_type)] = me
    outlook_dataset.to_netcdf(save_location)

Day 2 7
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008


In [131]:
save_location = 'data/outlooks/grid_outlooks.nc'
outlook_dataset.to_netcdf(save_location)

# Gridize reports

In [ ]:
# Create empty reports dataset: same coords and time, 3 hazard dimensions (and total dimension and T/F dimension?)
# iterate through reports (filtered to only be severe as in labelling), adding 1 to appropriate count

# Save files